In [5]:
### import 資料進來
import os, json, pandas as pd

base_dir = "/Users/Andypon/10_交大研究所/1141_01_機器學習與金融科技/data"

file_name = "mcc_codes.json"
file_path = os.path.join(base_dir, file_name)

with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 如果是 dict of scalar → 轉成 [{"code": k, "desc": v}, ...]
if isinstance(data, dict):
    mcc_codes_df = pd.DataFrame([{"code": k, "desc": v} for k, v in data.items()])
else:
    mcc_codes_df = pd.DataFrame(data)

print(mcc_codes_df.head())


file_name = "train_fraud_labels.json"
file_path = os.path.join(base_dir, file_name)

with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 如果是 dict of scalar → 轉成 [{"code": k, "desc": v}, ...]
if isinstance(data, dict):
    train_fraud_labels_df = pd.DataFrame([{"code": k, "desc": v} for k, v in data.items()])
else:
    train_fraud_labels_df = pd.DataFrame(data)

print(train_fraud_labels_df.head())

   code                                  desc
0  5812         Eating Places and Restaurants
1  5541                      Service Stations
2  7996  Amusement Parks, Carnivals, Circuses
3  5411          Grocery Stores, Supermarkets
4  4784                 Tolls and Bridge Fees
     code                                               desc
0  target  {'10649266': 'No', '23410063': 'No', '9316588'...


In [ ]:
#cars one heart encoding

SyntaxError: invalid syntax (ipython-input-3595165742.py, line 1)